In [1]:
api_key="2ddc2fe8-c965-480f-8224-53af0cdd5704"

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os 
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initilaize the pinecone 
pc=Pinecone(api_key=api_key)
#create the index 
if index_name not in pc.list_indexes().names():
    pc.create_index(index_name, 
                    dimension=768, #dimension of dense model 
                    metric='dotproduct',##sparse value supported for only dotproduct 
                    spec=ServerlessSpec(cloud='aws',region="us-east-1"))


In [7]:
index=pc.Index(index_name)
index

In [8]:
## vector embedding and sparsematrix 
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import GoogleGenerativeAIEmbeddings
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [9]:
from pinecone_text.sparse import BM25Encoder # used tfidf by default

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\puzan12\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [18]:
sentences=[
    "In the year 2020 i visited solu",
    "In the year 2021 i visited ilam",
    "In the year 2024 i visited Ranke",


]

In [19]:
bm25_encoder.fit(sentences)# applying tfidf 

100%|██████████| 3/3 [00:00<00:00, 3020.38it/s]


In [20]:
##stires the value in json 
bm25_encoder.dump("bm25_values.json")

In [21]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [22]:
retriever

PineconeHybridSearchRetriever(embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000028BFFD43E30>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000028BFFF07020>, index=<pinecone.data.index.Index object at 0x0000028BFB0AC1D0>)

In [23]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:00<00:00,  1.15it/s]


In [25]:
retriever.invoke("what city did i visit recent")

[Document(page_content='In the year 2024 i visited solu'),
 Document(page_content='In the year 2024 i visited Ranke'),
 Document(page_content='In the year 2021 i visited ilam'),
 Document(page_content='In the year 2020 i visited solu')]